In [1]:
import pandas as pd
from pathlib import Path

In [2]:
Catholic_file = 'data/data_train.csv'
df = pd.read_csv(Catholic_file)
df

,filename,category,class
0,0001-2.wav,healthy,0
1,0002-1.wav,healthy,0
2,0002-2.wav,healthy,0
3,0002-3.wav,healthy,0
4,0002-4.wav,healthy,0
...,...,...,...
206,0600-2.wav,wheezing,1
207,0600-3.wav,wheezing,1
208,0602-3.wav,wheezing,1
209,0603-1.wav,wheezing,1


In [3]:

df['relative_path'] = '/' + df['filename'].astype(str)
df = df[['relative_path', 'class']]
df.head()

,relative_path,class
0,/0001-2.wav,0
1,/0002-1.wav,0
2,/0002-2.wav,0
3,/0002-3.wav,0
4,/0002-4.wav,0


In [24]:
data_path = 'data/train'

In [5]:
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio

class Breath_sound_Util():
  
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    sig = torchaudio.functional.gain(sig, gain_db=5.0)
    sig = torchaudio.functional.dither(sig)
    sig = torchaudio.functional.lowpass_biquad(sig, sr, cutoff_freq=3000)
    sig = torchaudio.functional.highpass_biquad(sig, sr, cutoff_freq=2000)
    
    return (sig, sr)

  def resample(aud, newsr):
    sig, sr = aud
    
    if (sr == newsr):
     return aud

    num_channels = sig.shape[0]
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))
  

  def pad(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms
   
    if (sig_len > max_len):
      sig = sig[:,:max_len]
 
    elif (sig_len < max_len):

      repeated = []
      repeated.append(sig)
      required_len = max_len - sig_len

      while required_len > sig_len : 
        repeated.append(sig)
        require_len -= sig_len
      repeated.append(sig[:, :required_len])
 
      sig = torch.cat(repeated, 1)

    return (sig, sr)


  def time_shift(aud, shift_limit):
    sig,sr = aud
    _, sig_len = sig.shape
    shift_amt = int(random.random() * shift_limit * sig_len)
    return (sig.roll(shift_amt), sr)

  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    aud = torchaudio.transforms.TimeStretch(hop_length=2048, fixed_rate=5000000)
    top_db = 80
    
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

  def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec

    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
      aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
      aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

    return aug_spec
    print(aug_spec)

In [6]:
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio
import torchvision

class breathDS(Dataset):
    
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 48000
    self.shift_pct = 0.4
            
  def __len__(self):
    return len(self.df)    
    
  def __getitem__(self, idx):
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    class_id = self.df.loc[idx, 'class']
    aud = Breath_sound_Util.open(audio_file)
    reaud = Breath_sound_Util.resample(aud, self.sr)
    dur_aud = Breath_sound_Util.pad(reaud, self.duration)
    shift_aud = Breath_sound_Util.time_shift(dur_aud, self.shift_pct)
    sgram = Breath_sound_Util.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = Breath_sound_Util.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

    
    return aug_sgram, class_id

In [7]:
from torch.utils.data import random_split

brds = breathDS(df, data_path)

In [8]:

train_dl = torch.utils.data.DataLoader(brds, batch_size=16, shuffle=True)

In [9]:
Catholic_file1 = 'data/data_val.csv'
df1 = pd.read_csv(Catholic_file1)
df1

,filename,category,class
0,0041-1.wav,healthy,0
1,0041-2.wav,healthy,0
2,0041-3.wav,healthy,0
3,0041-4.wav,healthy,0
4,0042-1.wav,healthy,0
5,0042-2.wav,healthy,0
6,0042-3.wav,healthy,0
7,0042-4.wav,healthy,0
8,0043-1.wav,healthy,0
9,0043-2.wav,healthy,0


In [10]:
df1['relative_path'] = '/' + df1['filename'].astype(str)
df1 = df1[['relative_path', 'class']]
df1.head()

,relative_path,class
0,/0041-1.wav,0
1,/0041-2.wav,0
2,/0041-3.wav,0
3,/0041-4.wav,0
4,/0042-1.wav,0


In [11]:
data_path = 'data/val'

In [12]:
class breathDS1(Dataset):
    
  def __init__(self, df1, data_path):
    self.df1 = df1
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 48000
    self.shift_pct = 0.4
            
  def __len__(self):
    return len(self.df1)    
    
  def __getitem__(self, idx):
    audio_file = self.data_path + self.df1.loc[idx, 'relative_path']
    class_id = self.df1.loc[idx, 'class']
    aud = Breath_sound_Util.open(audio_file)
    reaud = Breath_sound_Util.resample(aud, self.sr)
    dur_aud = Breath_sound_Util.pad(reaud, self.duration)
    shift_aud = Breath_sound_Util.time_shift(dur_aud, self.shift_pct)
    sgram = Breath_sound_Util.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = Breath_sound_Util.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

    
    return aug_sgram, class_id

In [13]:
brds1 = breathDS1(df1, data_path)

In [14]:
val_dl = torch.utils.data.DataLoader(brds1, batch_size=16, shuffle=True)

In [15]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

# --------------------------------------------------------------
# 호흡음의 Healthy, Wheezing을 판단하는 Binary Classification Model
# --------------------------------------------------------------

class WhoWheezing(nn.Module):

    def __init__(self):
        super().__init__()
        conv_layers = []
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.dropout = nn.Dropout(0.4)
        self.lin = nn.Linear(in_features=64, out_features=2)
        self.dropout = nn.Dropout(0.4)
        self.sigmoid = nn.Sigmoid()

        self.conv = nn.Sequential(*conv_layers)
 
    def forward(self, x):
        x = self.conv(x)
        x = self.ap(x)
        x = x.view(x.shape[0], -1)
        x = self.lin(x)
        x = self.dropout(x)
        x = self.sigmoid(x)
        return x

Model1 = WhoWheezing()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model1 = Model1.to(device)
next(Model1.parameters()).device

device(type='cuda', index=0)

In [16]:
from torchcontrib.optim import SWA

In [17]:
from sklearn.metrics import confusion_matrix
import numpy as np
import sklearn.metrics as metrics


def training(model, train_dl, num_epochs):
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0

    for i, data in enumerate(train_dl):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
        _, prediction = torch.max(outputs,1)
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    print(f'Epoch: {epoch}, Loss: {avg_loss:.4f}, Accuracy: {acc:.4f}')

  print('Finished Training')
    
num_epochs=30
training(Model1, train_dl, num_epochs)

Epoch: 0, Loss: 0.6933, Accuracy: 0.5498
Epoch: 1, Loss: 0.6345, Accuracy: 0.8483
Epoch: 2, Loss: 0.5857, Accuracy: 0.8389
Epoch: 3, Loss: 0.5617, Accuracy: 0.8389
Epoch: 4, Loss: 0.5604, Accuracy: 0.8436
Epoch: 5, Loss: 0.5231, Accuracy: 0.9005
Epoch: 6, Loss: 0.5114, Accuracy: 0.9194
Epoch: 7, Loss: 0.5185, Accuracy: 0.8957
Epoch: 8, Loss: 0.4993, Accuracy: 0.9052
Epoch: 9, Loss: 0.5065, Accuracy: 0.8910
Epoch: 10, Loss: 0.5116, Accuracy: 0.8957
Epoch: 11, Loss: 0.4932, Accuracy: 0.9194
Epoch: 12, Loss: 0.5077, Accuracy: 0.8768
Epoch: 13, Loss: 0.4882, Accuracy: 0.9052
Epoch: 14, Loss: 0.4922, Accuracy: 0.9289
Epoch: 15, Loss: 0.4933, Accuracy: 0.8957
Epoch: 16, Loss: 0.4873, Accuracy: 0.9194
Epoch: 17, Loss: 0.4740, Accuracy: 0.8910
Epoch: 18, Loss: 0.4825, Accuracy: 0.9242
Epoch: 19, Loss: 0.5000, Accuracy: 0.9147
Epoch: 20, Loss: 0.4614, Accuracy: 0.9336
Epoch: 21, Loss: 0.4754, Accuracy: 0.9147
Epoch: 22, Loss: 0.4934, Accuracy: 0.9005
Epoch: 23, Loss: 0.4806, Accuracy: 0.9194
Ep

In [182]:
def inference (model, val_dl):
  correct_prediction = 0
  total_prediction = 0

  with torch.no_grad():
    for data in val_dl:
      
      inputs, labels = data[0].to(device), data[1].to(device)
      inputs_m, inputs_s = inputs.mean(), inputs.std()
      inputs = (inputs - inputs_m) / inputs_s
      outputs = model(inputs)
      _, prediction = torch.max(outputs,1)
      
      correct_prediction += (prediction == labels).sum().item()
      total_prediction += prediction.shape[0]
    
  acc = correct_prediction/total_prediction
  print(f'Accuracy: {acc:.4f}, Total items: {total_prediction}')
  
inference(Model1, val_dl)

Accuracy: 0.6923, Total items: 52


In [17]:
import numpy as np
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F

from skorch import NeuralNetClassifier


X, y = make_classification(1000, 20 n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)

class WhoWheezing(nn.Module):

    def __init__(self):
        super().__init__()
        conv_layers = []
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.dropout = nn.Dropout(0.4)
        self.lin = nn.Linear(in_features=64, out_features=2)
        self.dropout = nn.Dropout(0.4)
        self.sigmoid = nn.Sigmoid()

        self.conv = nn.Sequential(*conv_layers)
 
    def forward(self, x):
        x = self.conv(x)
        x = self.ap(x)
        x = x.view(x.shape[0], -1)
        x = self.lin(x)
        x = self.dropout(x)
        x = self.sigmoid(x)
        return x


net = NeuralNetClassifier(
    Model1,
    max_epochs=100,
    lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

net.fit(X, y)
y_proba = net.predict_proba(X)
y_proba = net.predict_proba(X)

SyntaxError: invalid syntax (<ipython-input-17-578dc92b27d0>, line 9)

In [26]:
# using the example from the README
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from torch import nn
import torch.nn.functional as F
from torch.utils.data.dataloader import default_collate

from skorch import NeuralNetClassifier
from skorch.dataset import Dataset
from skorch.dataset import CVSplit

X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)

'''class MyModule(nn.Module):
    def __init__(self, num_units=10, nonlin=F.relu):
        super(MyModule, self).__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X'''

class WhoWheezing(nn.Module):

    def __init__(self):
        super().__init__()
        conv_layers = []
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.dropout = nn.Dropout(0.4)
        self.lin = nn.Linear(in_features=64, out_features=2)
        self.dropout = nn.Dropout(0.4)
        self.sigmoid = nn.Sigmoid()

        self.conv = nn.Sequential(*conv_layers)
 
    def forward(self, x):
        x = self.conv(x)
        x = self.ap(x)
        x = x.view(x.shape[0], -1)
        x = self.lin(x)
        x = self.dropout(x)
        x = self.sigmoid(x)
        return x

# pack the data into a Dataset
'''class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, i):
        Xi = self.X[i]
        yi = self.y[i]
        return self.transform(Xi, yi)'''

class breathDS(Dataset):
    
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 48000
    self.shift_pct = 0.4
            
  def __len__(self):
    return len(self.df)    
    
  def __getitem__(self, idx):
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    class_id = self.df.loc[idx, 'class']
    aud = Breath_sound_Util.open(audio_file)
    reaud = Breath_sound_Util.resample(aud, self.sr)
    dur_aud = Breath_sound_Util.pad(reaud, self.duration)
    shift_aud = Breath_sound_Util.time_shift(dur_aud, self.shift_pct)
    sgram = Breath_sound_Util.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = Breath_sound_Util.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

    return self.transform(aug_sgram, class_id)
    #return aug_sgram, class_id

#ds = MyDataset(df1, data_path)
ds = breathDS1(df, data_path)

class SliceDatasetX(Dataset):
    """Helper class that wraps a torch dataset to make it work with sklearn"""
    def __init__(self, dataset, collate_fn=default_collate):
        self.dataset = dataset
        self.collate_fn = collate_fn

        self._indices = list(range(len(self.dataset)))
        
    def __len__(self):
        return len(self.dataset)
        
    @property
    def shape(self):
        return len(self),
    
    def __getitem__(self, i):
        if isinstance(i, (int, np.integer)):
            Xb = self.transform(*self.dataset[i])[0]
            return Xb
        
        if isinstance(i, slice):
            i = self._indices[i]

        Xb = self.collate_fn([self.transform(*self.dataset[j])[0] for j in i])
        return Xb
    

params = {
    'lr': [0.001, 0.0001, 0.00001],
    'max_epochs': [30, 40, 50, 60, 70, 80],
    'batch_size': [4, 8, 16, 32, 64]
}

net = NeuralNetClassifier(
    WhoWheezing,
    max_epochs=50,
    lr=0.001,
    batch_size=16,
    iterator_train__shuffle=True,
    verbose=0,
    train_split=None
)


In [27]:
len(ds)

211

In [28]:
gs = GridSearchCV(net, params, refit=False, cv=3, scoring='accuracy')

# we have to extract the target data, otherwise sklearn will complain
y_from_ds = np.asarray([ds[i][1] for i in range(len(ds))])
# wrap the dataset into the new helper class
ds_sliceable = SliceDatasetX(ds)

gs.fit(ds_sliceable , y_from_ds)

GridSearchCV(cv=3,
             estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.WhoWheezing'>,
),
             param_grid={'batch_size': [4, 8, 16, 32, 64],
                         'lr': [0.001, 0.0001, 1e-05],
                         'max_epochs': [30, 40, 50, 60, 70, 80]},
             refit=False, scoring='accuracy')

In [29]:
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_lr,param_max_epochs,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,6.392112,0.019546,0.016569,0.000539,4,0.001,30,"{'batch_size': 4, 'lr': 0.001, 'max_epochs': 30}",0.802817,0.914286,0.957143,0.891415,0.065046,7
1,7.750510,0.095295,0.016684,0.000503,4,0.001,40,"{'batch_size': 4, 'lr': 0.001, 'max_epochs': 40}",0.760563,0.914286,0.971429,0.882093,0.089044,11
2,9.275710,0.130940,0.016574,0.000424,4,0.001,50,"{'batch_size': 4, 'lr': 0.001, 'max_epochs': 50}",0.774648,0.957143,0.971429,0.901073,0.089586,5
3,10.738872,0.102512,0.016618,0.000303,4,0.001,60,"{'batch_size': 4, 'lr': 0.001, 'max_epochs': 60}",0.774648,0.942857,1.000000,0.905835,0.095652,3
4,12.107062,0.111557,0.016833,0.000645,4,0.001,70,"{'batch_size': 4, 'lr': 0.001, 'max_epochs': 70}",0.788732,0.985714,1.000000,0.924816,0.096402,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,6.519905,0.094682,0.014494,0.000225,64,0.00001,40,"{'batch_size': 64, 'lr': 1e-05, 'max_epochs': 40}",0.591549,0.328571,0.728571,0.549564,0.165976,69
86,7.543148,0.085940,0.014163,0.000325,64,0.00001,50,"{'batch_size': 64, 'lr': 1e-05, 'max_epochs': 50}",0.647887,0.771429,0.128571,0.515962,0.278531,73
87,8.653281,0.044723,0.014388,0.000407,64,0.00001,60,"{'batch_size': 64, 'lr': 1e-05, 'max_epochs': 60}",0.563380,0.885714,0.271429,0.573508,0.250883,67
88,9.594540,0.095816,0.014315,0.000268,64,0.00001,70,"{'batch_size': 64, 'lr': 1e-05, 'max_epochs': 70}",0.323944,0.814286,0.871429,0.669886,0.245728,60


In [50]:
del gs

In [ ]:
model = DecisionTreeClassifier()
# parameter 넣어줄 값들 dict 형태로 정의해주기
h_para = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

grid_dtree = GridSearchCV(model, param_grid=h_para,
                         cv=5, refit=True, return_train_score=True)
# GridSearchCV 인자설명
# cv = 하나의 파라미터 쌍으로 모델링할 때 train, test 교차검증을 3번실시하겠다는 뜻
# refit=True : GridSearch한 후 가장 최고로 좋은 파라미터로 학습시켜 놓겠다.
# ㄴ> 이것 때문에 애초에 GridSearchCV 적용한 객체만으로 최적의 파라미터 적용된 모델로드 가능

# GridSearch 하면서 모든 파라미터값들에 대해 학습 수행
grid_dtree.fit(train_x, train_y)

# 각 파라미터값들에 대한 모델 결과값들이 cv_results_ 객체에 할당됨
scores_df = pd.DataFrame(grid_dtree.cv_results_)

# score 결과값(ndarray형태로 할당됨) 중 특정 칼럼들만 가져오기 
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

In [40]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

config = {
    "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}

NameError: name 'partial' is not defined